In [1]:
import os
import pandas as pd
import random

In [2]:
#function definitions

def isNaN(num):
    return num != num

def proteinSetOfPhageSet(phageset):
    my_protein_clusters = dict()
    mark_for_write = False
    count_X = 0
    count_proteins = 0
    with open(GPD_proteins) as seq_file:
        for line in seq_file:
            if line.startswith(">"):
                identifier = line.strip()[1:]
                phageID = "_".join(identifier.split("_")[0:2])
                if phageID in phageset:
                    mark_for_write = True
                else:
                    mark_for_write = False
            else:
                if mark_for_write:
                    if "X" not in line:
                        count_proteins += 1
                        if identifier in protein_clusters:
                            cluster_name = protein_clusters[identifier]
                        else:
                            cluster_name = identifier
                        if cluster_name not in my_protein_clusters:
                            my_protein_clusters[cluster_name] = []
                        my_protein_clusters[cluster_name].append({"proteinID":identifier,"seq":line, "size":len(line)})

                    else:
                        count_X = count_X +1

    print(count_proteins, "proteins found in", len(my_protein_clusters), "clusters, and additional",count_X,"proteins countain X")
    return(my_protein_clusters)

def tileProtSeqWOverlap(aa_seq, tilesize, overlap = -1):
    # start tiles from both side, let "middle tile" over lap with already existing tiles
    # start second overlapping set from the middle in both directions to the side
    # ____   ___   ____ #
    #    ____   ____  #
    if overlap == -1:
        overlap = int(tilesize/2)
    tiles = []
    center = int(len(aa_seq)/2)
    seq = aa_seq[:center]
    pos = 0
    
    if len(aa_seq) >= tilesize:
        while len(seq) >= tilesize:
            tiles.append((seq[:tilesize],pos))
            seq = seq[tilesize-overlap:]
            pos = pos + tilesize - overlap
        tiles.append((aa_seq[pos:pos+tilesize],pos))
        seq = aa_seq[center:]
        pos = len(aa_seq) - tilesize
        while len(seq) >= tilesize:
            tiles.append((seq[-tilesize:],pos))
            seq = seq[:-(tilesize-overlap)]
            pos = pos-tilesize+overlap   
        last_tile = (aa_seq[pos:pos+tilesize],pos)
        if aa_seq[pos:pos+tilesize] != "" and last_tile not in tiles:
            tiles.append(last_tile)
    else:
        tiles.append((aa_seq, -1))
        
    return(tiles)

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches
        
def find_all_proteins_in_str(long_string):
    idxs = list(find_all(long_string, 'vig')) 
    prots = set()
    for i in idxs:
        t = long_string[i-1:]
        ps= "_".join(t.split("_")[0:3]).split("'")[0].split("pos")[0]
        prots.add(ps)
    return prots

# GPD Analysis - Count Phages and Proteins

Download GPD files from

Luis F. Camarillo-Guerrero, Alexandre Almeida, Guillermo Rangel-Pineros, Robert D. Finn, Trevor D. Lawley,
Massive expansion of human gut bacteriophage diversity,
Cell, 2021, https://doi.org/10.1016/j.cell.2021.01.029.

affiliated data:
http://ftp.ebi.ac.uk/pub/databases/metagenomics/genome_sets/gut_phage_database/

In [3]:
protein_clusters_file ="../../PCs_GPD.txt"
protein_clusters = dict()
with open(protein_clusters_file) as myfile:
    for line in myfile.readlines():
        line = line.strip()
        rep = line.split("\t")[0]
        for phID in line.split("\t"):
            protein_clusters[phID] = rep
            
GPD_proteins = "../../GPD_proteome.faa"
phage_meta_df = pd.read_table("../../GPD_phage_metadata.csv", sep = ",", index_col = 0)

In [4]:
my_phages = set()
for index, row in phage_meta_df.iterrows():
    my_phages.add(index)

print("All Phages in GPD", len(my_phages))

All Phages in GPD 142810


In [5]:
_ = proteinSetOfPhageSet(my_phages)

7581279 proteins found in 309842 clusters, and additional 528 proteins countain X


In [6]:
my_phages = set()
for index, row in phage_meta_df.iterrows():
    if row["checkV_MIUViG"] == "High-quality":
        if not isNaN(row["Continents_detected"]):
            my_phages.add(index)

print("HQ (checkV_MIUViG = 'High-quality') Phages with at least 1 detected Sample", len(my_phages))

HQ (checkV_MIUViG = 'High-quality') Phages with at least 1 detected Sample 38848


In [7]:
my_phages = set()
for index, row in phage_meta_df.iterrows():
    if row["checkV_MIUViG"] == "High-quality":
        my_phages.add(index)

print("HQ (checkV_MIUViG = 'High-quality') Phages:", len(my_phages))

HQ (checkV_MIUViG = 'High-quality') Phages: 41427


In [12]:
my_phages = set()
for index, row in phage_meta_df.iterrows():
    if row["checkV_MIUViG"] == "High-quality":
        if not isNaN(row["Continents_detected"]):
            countrylist = row["Continents_detected"].split(",") 
            if 'North America' in countrylist :
                if countrylist.count("North America") > 0 :
                    my_phages.add(index)

print("HQ (checkV_MIUViG = 'High-quality') Phages with at least 1 North American Sample", len(my_phages))

HQ (checkV_MIUViG = 'High-quality') Phages with at least 1 North American Sample 7614


In [13]:
phageome_clusters = proteinSetOfPhageSet(my_phages)

588783 proteins found in 84306 clusters, and additional 47 proteins countain X


In [14]:
my_phages = set()
for index, row in phage_meta_df.iterrows():
    if row["checkV_MIUViG"] == "High-quality":
        if not isNaN(row["Continents_detected"]):
            countrylist = row["Continents_detected"].split(",") 
            if 'North America' in countrylist :
                if countrylist.count("North America") > 50 :
                    my_phages.add(index)

print("HQ (checkV_MIUViG = 'High-quality') prevalent (>50 samples) North American Phages", len(my_phages))

HQ (checkV_MIUViG = 'High-quality') prevalent (>50 samples) North American Phages 112


In [15]:
phageome_pilot_clusters = proteinSetOfPhageSet(my_phages)

8023 proteins found in 3354 clusters, and additional 6 proteins countain X


In [17]:
protein_seq_file = "../data/PhageScan_pilot_proteinClusterReps.faa"
my_protein_clusters = phageome_pilot_clusters

# For "FULL" design ("theoretical design" in Manuscript) uncomment the following two lines
#write_file_name = "PhageScan_proteinClusterReps.faa"
#my_protein_clusters = phageome_clusters

In [14]:
with open(protein_seq_file,'w') as write_file:
    for cluster in my_protein_clusters.values():
        identifier = ""
        longest = 0
        for protein in cluster:
            identifier += protein["proteinID"] +"_"
            if protein["size"]>longest:
                longest = protein["size"]
                representative_seq = protein["seq"]
                repID = protein["proteinID"]
                
        write_file.write(">REP_"+repID+"_ALL_"+identifier[:-1] + "\n")
        write_file.write(representative_seq)

In [18]:
l = 0
with open(protein_seq_file) as seq_file:
    for line in seq_file:
        if not line.startswith(">"):
            seq = line[:-1]
            l+=len(seq)
print("length of all proteins together:",l,"aa")

length of all proteins together: 750776 aa


# Simple tiling approach

In [15]:
tilesize = 56
overlap = int(tilesize/2)

pepsyn_all_tiles = dict()

with open(protein_seq_file) as rep_file:
    for line in rep_file:
        if not line.startswith(">"):
            seq = line.strip()
            tile_vec = tileProtSeqWOverlap(seq, tilesize, overlap)
            for tile, pos in tile_vec:
                if tile not in pepsyn_all_tiles:
                    pepsyn_all_tiles[tile] = []
                pepsyn_all_tiles[tile].append(str(protein)+"pos_"+str(pos))
        else:
            protein = line.strip()[1:]
                
print(len(pepsyn_all_tiles),"unique tiles")
no_pepsyn_tiles = len(pepsyn_all_tiles)

23745 unique tiles


# Dolphyn tyling approach
check out notebook Dolphyn_library_design.ipynb for more guidance on using Dolphyn

In [10]:
import sys
sys.path.insert(0, os.path.abspath('../dolphyn'))
import dolphyn as D

In [ ]:
# takes time (~25min on my laptop), uncomment to run!
ge = D.findEpitopes(testrun = 0, protein_seq_file = protein_seq_file, epitile_size=15)
dolphyn_tiles, _ = D.peptideStitching(no_epis_per_tile = 3, linker_seq = "GGGGS" , global_epitopes = ge, return_unused_epis = False)


# Build final library

In [12]:
prot_file = "phageScan_proteins.faa"
dna_file_noadap = "phageome_peptides_oligos_noadap.fasta"
dna_file = "phageScan_peptides_oligos.fasta"

five_prime_adapter = "AGGAATTCCGCTGCGT"
three_prime_adapter = "ATGGTCACAGCTGTGC"

In [16]:
anno = dict()
AMINOACIDS = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
def junkseq(length):
    return(("***" + "".join(random.choice(AMINOACIDS) for i in range(length)))[:length])

with open(prot_file, "w") as pf:
    for seq, tile in dolphyn_tiles.items():
        n = str(tile["protein set"])+"_"+str(tile["tile number"])+"of"+str(tile["tiles in protein(set)"])
        h = abs(hash(n))
        anno["dolphyn_"+str(h)] = "dolphyn_"+n
        pf.write(">"+"dolphyn_"+str(h)+"\n")
        pf.write(seq+"*\n")
        
    for seq, prots in pepsyn_all_tiles.items():
        n = str(prots)
        h = abs(hash(n))
        anno["pepsyn_"+str(h)] = "pepsyn_"+n
        pf.write(">"+"pepsyn_"+str(h)+"\n")
        pf.write(seq + junkseq(56-len(seq))+"\n")
        
    for seq, tile in ge.items():
        n = str(tile["proteins"])+"_"+str(tile["start_pos"])+"_"+str(tile["probability"])
        h = abs(hash(n))
        anno["dolphynepitopes_"+str(h)] = "dolphynepitopes_"+n
        pf.write(">"+"dolphynepitopes_"+str(h)+"\n")
        pf.write("GGGGS" + seq+ junkseq(36) + "\n")

In [17]:
revtrans_command = "pepsyn revtrans "+ prot_file + " " + dna_file_noadap
stream = os.popen(revtrans_command)
output = stream.read().strip()
output

'pepsyn revtrans phageScan_proteins.faa phageome_peptides_oligos_noadap.fasta'

In [19]:
with open(dna_file_noadap, "r") as na:
    with open(dna_file, "w") as wa:
        for line in na:
            l = line.strip()
            if not line.startswith(">"):
                l = five_prime_adapter + l + three_prime_adapter
            wa.write(l+"\n")

In [20]:
dolp = 0
epis = 0
peps = 0
total = 0
with open(dna_file, "r") as wa:
    for line in wa:
        total +=1
        if line.startswith(">dolphyn_"):
            dolp += 1
        elif line.startswith(">dolphynepitopes_"):
            epis += 1
        elif line.startswith(">pepsyn_"):
            peps += 1
            
print("total:", total/2 , "dolphyn:", dolp, "epitopes:", epis, "pepsyn:", peps)

total: 23758.0 dolphyn: 3 epitopes: 10 pepsyn: 23745


# Annotations

In [21]:
annoall = pd.DataFrame(anno.items(), columns = ["tile_id","allinfo"])
annoall.index = annoall["tile_id"]
annoall["library"] = [el[0] for el in annoall['tile_id'].str.split("_")]
annoall["AA"] = ""
annoall["oligo"] = ""
identifier = ""
with open(dna_file_noadap, "r") as wa:
    for line in wa:
        if line.startswith(">"):
            identifier = line.strip()[1:]
        else:
            annoall.loc[identifier, "oligo"] = line.strip()
identifier = ""
with open(prot_file, "r") as wa:
    for line in wa:
        if line.startswith(">"):
            identifier = line.strip()[1:]
        else:
            annoall.loc[identifier, "AA"] = line.strip()

annoall.drop("tile_id", axis = 1, inplace=True)

In [22]:
position_dict = dict()
for allinfo in annoall[annoall.library=='pepsyn'].allinfo:
    splits = allinfo.split("pos_")
    if len(splits) == 3:
        protein = splits[0].split("REP_")[1].split("_ALL_")[0]
        pos = pd.to_numeric(splits[1].split("'")[0])
        if protein not in position_dict:
            position_dict[protein] = []
        position_dict[protein].append(pos)
        protein = splits[1].split("', 'REP_")[1].split("_ALL_")[0]
        pos = pd.to_numeric(splits[2].split("'")[0])
        if protein not in position_dict:
            position_dict[protein] = []
        position_dict[protein].append(pos)
    else:
        protein = splits[0].split("REP_")[1].split("_ALL_")[0]
        pos = pd.to_numeric(splits[1].split("'")[0])
        if protein not in position_dict:
            position_dict[protein] = []
        position_dict[protein].append(pos)

for k in position_dict:
    position_dict[k].sort()

In [23]:
position_dict_epis = dict()
for allinfo in annoall[annoall.library=='dolphynepitopes'].allinfo:
    splits = allinfo.split("REP_")
    no_prot = len(splits)-1
    if no_prot > 0:
        #print(allinfo)
        for ix in range(no_prot):
            protein = splits[ix+1].split("_ALL")[0]
            pos = pd.to_numeric(splits[-1].split("[")[1].split("]")[0].split(", ")[ix])
            if protein not in position_dict_epis:
                position_dict_epis[protein] = []
            position_dict_epis[protein].append(pos)
            
for k in position_dict_epis:
    position_dict_epis[k].sort()

In [24]:
proteinseqs = dict()
with open(protein_seq_file) as seq_file:
    for line in seq_file:
        if line.startswith(">"):
            identifier = line.strip()[5:].split("_ALL_")[0]
            #print(identifier)
        else:
            if identifier in proteinseqs:
                print("WARNING! Identifier already exists", identifier)
            else:
                proteinseqs[identifier] = line.strip()

In [25]:
inhouse_annotation_standard = ['u_pep_id', 'pep_id', 'pro_id', 'pep_rank', 'pos_start', 'pos_end',
       'UniProt_acc', 'pep_dna', 'pep_aa', 'pro_len', 'pro_motifs', 'GO',
       'RefSeq', 'taxon_id', 'taxon_species', 'taxon_genus', 'gene_symbol',
       'gene_synonym', 'product', 'description', 'Pfam', 'EMBL', 'InterPro']

In [26]:
anno = pd.DataFrame(columns = inhouse_annotation_standard + ["pro_id_all","Host_range_taxon","sublibrary"])

In [27]:
anno.u_pep_id = "DolphynLa_001_" + annoall.oligo.str.slice(0,50)
anno.pep_id = annoall.index
anno.pep_dna = annoall.oligo
anno.pep_aa = annoall.AA
anno.sublibrary = annoall.library
#dolphyn tiles contain three epitopes and therefore no start position
anno.pro_id[annoall.index.str.startswith("dolphyn_")] = annoall.allinfo[annoall.index.str.startswith("dolphyn_")].str.split("_ALL_").str[0].str.slice(14)
anno.pep_rank[annoall.index.str.startswith("dolphyn_")] = annoall.allinfo[annoall.index.str.startswith("dolphyn_")].str.split("of").str[0].str.split("_").str[-1]
#dolphyn epitopes may appear in more than one protein
anno.pro_id[annoall.index.str.startswith("dolphynepi")] = annoall.allinfo[annoall.index.str.startswith("dolphynepi")].str.split("_ALL").str[0].str.slice(22)
anno.pos_start[annoall.index.str.startswith("dolphynepi")] = pd.to_numeric(annoall.allinfo[annoall.index.str.startswith("dolphynepi")].str.split("[").str[2].str.split(",").str[0].str.split("]").str[0])
anno.pos_end[annoall.index.str.startswith("dolphynepi")] = pd.to_numeric(annoall.allinfo[annoall.index.str.startswith("dolphynepi")].str.split("[").str[2].str.split(",").str[0].str.split("]").str[0])+15
anno.pep_rank[(anno.sublibrary == "dolphynepitopes")] = anno[(anno.sublibrary == "dolphynepitopes")].apply(lambda x: position_dict_epis[x.pro_id].index(x.pos_start), axis = 1) + 1
#pepsyn tiles MAY occur in more than one protein
anno.pro_id[annoall.index.str.startswith("pep")] = annoall.allinfo[annoall.index.str.startswith("pep")].str.split("_ALL").str[0].str.slice(13)
anno.pos_start[annoall.index.str.startswith("pep")] = pd.to_numeric(annoall.allinfo[annoall.index.str.startswith("pep")].str.split("pos_").str[1].str.split("'").str[0])
anno.pos_end[annoall.index.str.startswith("pep")] = pd.to_numeric(annoall.allinfo[annoall.index.str.startswith("pep")].str.split("pos_").str[1].str.split("'").str[0]) + 56
anno.pep_rank[(anno.sublibrary == "pepsyn")&(~anno.pro_id.isna())&(~anno.pos_start.isna())] = anno[(anno.sublibrary == "pepsyn")&(~anno.pro_id.isna())&(~anno.pos_start.isna())].apply(lambda x: position_dict[x.pro_id].index(x.pos_start), axis = 1) + 1
#all proteins start with ivig or uvig
anno.pro_id_all[annoall.index.str.startswith("dolphyn")] = annoall.allinfo[annoall.index.str.startswith("dolphyn")].apply(find_all_proteins_in_str)
anno.pro_id_all[annoall.index.str.startswith("pep")] = annoall.allinfo[annoall.index.str.startswith("pep")].apply(find_all_proteins_in_str)
anno.pro_len[~anno.pro_id.isna()] = anno.pro_id[~anno.pro_id.isna()].apply(lambda x: len(proteinseqs[x]))
anno.taxon_id[~anno.pro_id.isna()] = anno.pro_id[~anno.pro_id.isna()].str.split("_").str[:2].apply(lambda x: "_".join(x))
anno.taxon_species[~anno.taxon_id.isna()] = phage_meta_df.loc[anno.taxon_id[~anno.taxon_id.isna()],"Predicted_phage_taxon"]
anno.Host_range_taxon[~anno.taxon_id.isna()] = phage_meta_df.loc[anno.taxon_id[~anno.taxon_id.isna()],"Host_range_taxon"]
# use only first protein for pep_rank
# pepsyn repeated tiles: pepsyn_488986999078171116,pepsyn_7965792066393306560,pepsyn_8667934916366736320,pepsyn_9099708212137372566


In [28]:
#anno.to_csv("PhageScan_PeptideAnno.tsv",sep="\t")

#### be aware that different versions may alter the random state for balancing training data and the random forest itself and therefore lead to differently chosen peptides